In [1]:
import os
import sys
import json
import numpy as np
from tqdm import tqdm
sys.path.append("..")

from src.movientities.models.lstm import BiLSTM
from src.movientities.utils.processors import NerPreProcessor
from src.movientities.bilstm_crf_trainer import TrainerBiLstm
from src.movientities.data.build_dataset import Corpus, BuildData

In [2]:
# Create datasets with tab as splitter for corpus of torch text to handle
BuildData.create_finaldata("/home/karaz/Desktop/MovieEntityRecognizer/data/raw/mitmovies/engtest.bio", "/home/karaz/Desktop/MovieEntityRecognizer/data/modified/mitmovies_tab_format/test.txt", splits="\t")
BuildData.create_finaldata("/home/karaz/Desktop/MovieEntityRecognizer/data/raw/mitmovies/engtest.bio", "/home/karaz/Desktop/MovieEntityRecognizer/data/modified/mitmovies_tab_format/test.txt", splits="\t")

In [5]:
dataset = Corpus(
    input_folder="/home/karaz/Desktop/MovieEntityRecognizer/data/modified/mitmovies_tab_format",
    min_word_freq=3,  # any words occurring less than 3 times will be ignored from vocab
    batch_size=64)
print(f"Train set: {len(dataset.train_dataset)} sentences")
print(f"Test set: {len(dataset.test_dataset)} sentences")

Train set: 9775 sentences
Test set: 2443 sentences


In [11]:
model = BiLSTM(
    input_dim=len(dataset.word_field.vocab),
    embedding_dim=300,
    hidden_dim=64,
    output_dim=len(dataset.tag_field.vocab),
    lstm_layers=2,
    emb_dropout=0.5,
    lstm_dropout=0.1,
    fc_dropout=0.25,
    word_pad_idx=dataset.word_pad_idx
)
# Initialize weights and embeddings
model.init_weights()
model.init_embeddings(word_pad_idx=dataset.word_pad_idx)
print(f"The model has {model.count_parameters():,} trainable parameters.")
print(model)

The model has 963,274 trainable parameters.
BiLSTM(
  (embedding): Embedding(2244, 300, padding_idx=1)
  (emb_dropout): Dropout(p=0.5, inplace=False)
  (lstm): LSTM(300, 64, num_layers=2, dropout=0.1, bidirectional=True)
  (fc_dropout): Dropout(p=0.25, inplace=False)
  (fc): Linear(in_features=128, out_features=26, bias=True)
)


In [14]:
from torch.optim import Adam
from torch import nn
ner = TrainerBiLstm(
  model=model,
  data=dataset,
  optimizer_cls=Adam,
  loss_fn_cls=nn.CrossEntropyLoss
)
ner.train(10)

Epoch: 01 | Epoch Time: 0m 9s
	Trn Loss: 1.469 | Trn Acc: 65.36%


AttributeError: 'Corpus' object has no attribute 'val_iter'

In [5]:
model = BiLSTM_CRF(vocab_size, label_map, vocab_size, hidden_layer_size)
trainer = Trainer_BiLstm(vocab_size, hidden_layer_size, tokenizer, label_map, dataset, epochs=10, batch_size=16, model=model, logdir="BiLSTM_logs_default")
trainer.train()

 10%|▉         | 963/9775 [00:57<08:45, 16.76it/s]


KeyboardInterrupt: 